# Day 2 - Classifying embeddings with Keras and the Gemini API
## 概述

歡迎回來參加 Kaggle 5 天生成式 AI 課程！在本筆記本中，你將學習如何利用 Gemini API 產生的嵌入向量來訓練一個模型，該模型可以根據文章內容將 newsgroup 貼文分類到相應的分類中。

這種技術以 Gemini API 的嵌入向量作為輸入，不需直接對文字進行訓練，因此與從頭訓練文字模型相比，即便使用較少的範例，也能取得不錯的表現。

## 幫助

**常見問題請參考** [FAQ 與故障排除指南](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs)。


In [ ]:
!pip uninstall -qqy jupyterlab kfp 2>/dev/null  # 移除未使用且可能衝突的套件
!pip install -U -q "google-genai==1.7.0"


In [2]:
from google import genai
from google.genai import types

genai.__version__


'1.7.0'

### 設定 API 金鑰

請先將 `YOUR_GOOGLE_API_KEY` 替換成自己的 API 金鑰。  
若你還沒有 API 金鑰，可前往 [AI Studio](https://aistudio.google.com/app/apikey) 取得，詳情請參考 [Gemini API 文件](https://ai.google.dev/gemini-api/docs/api-key)。


In [ ]:
client = genai.Client(api_key='YOUR_GOOGLE_API_KEY')

## 資料集

[20 Newsgroups 文本資料集](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) 包含 18,000 篇 newsgroup 貼文，分為 20 個主題，並以訓練集和測試集進行劃分。此教學將使用訓練集和測試集的抽樣子集，並使用 Pandas 進行部分處理。


In [5]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")

# 查看資料集中的分類名稱
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

這是一筆訓練資料的範例：

In [6]:
print(newsgroups_train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







接下來，我們先將資料預處理，將每則訊息中的主旨與內文提取出來，藉此移除姓名與電子信箱等敏感資訊。這是一個可選的步驟，可將原始電子郵件貼文轉換成較通用的文字，方便在其他情境中使用。


In [7]:
import email
import re
import pandas as pd

def preprocess_newsgroup_row(data):
    # 只擷取主旨與內文
    msg = email.message_from_string(data)
    text = f"{msg['Subject']}\n\n{msg.get_payload()}"
    # 移除任何殘留的電子郵件地址
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    # 將每筆資料截斷為最多 5000 個字元
    text = text[:5000]
    return text

def preprocess_newsgroup_data(newsgroup_dataset):
    # 將資料點放入 DataFrame 中
    df = pd.DataFrame({"Text": newsgroup_dataset.data, "Label": newsgroup_dataset.target})
    # 清理文字資料
    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)
    # 將標籤轉換為分類名稱
    df["Class Name"] = df["Label"].map(lambda l: newsgroup_dataset.target_names[l])
    return df

In [8]:
# 對訓練與測試資料集進行預處理
df_train = preprocess_newsgroup_data(newsgroups_train)
df_test = preprocess_newsgroup_data(newsgroups_test)

df_train.head()


,Text,Label,Class Name
0,WHAT car is this!?\n\n I was wondering if anyo...,7,rec.autos
1,SI Clock Poll - Final Call\n\nA fair number of...,4,comp.sys.mac.hardware
2,"PB questions...\n\nwell folks, my mac plus fin...",4,comp.sys.mac.hardware
3,Re: Weitek P9000 ?\n\nRobert J.C. Kyanko () wr...,1,comp.graphics
4,Re: Shuttle Launch Question\n\nFrom article <>...,14,sci.space


接著，我們將抽樣部份資料：從訓練資料集中取 100 筆資料，並篩選只保留包含 "sci"（科學相關）的分類，以進行本次教學。


In [9]:
def sample_data(df, num_samples, classes_to_keep):
    # 對每個標籤抽取 num_samples 筆資料
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)
    )
    df = df[df["Class Name"].str.contains(classes_to_keep)]
    # 因為只保留了部分分類，重新編碼標籤
    df["Class Name"] = df["Class Name"].astype("category")
    df["Encoded Label"] = df["Class Name"].cat.codes
    return df

In [10]:
TRAIN_NUM_SAMPLES = 100
TEST_NUM_SAMPLES = 25
# 保留分類中包含 'sci' 的（科學相關）
CLASSES_TO_KEEP = "sci"

df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)

In [11]:
df_train.value_counts("Class Name")


Class Name
sci.crypt          100
sci.electronics    100
sci.med            100
sci.space          100
Name: count, dtype: int64

In [12]:
df_test.value_counts("Class Name")

Class Name
sci.crypt          25
sci.electronics    25
sci.med            25
sci.space          25
Name: count, dtype: int64

## 產生嵌入向量

在這個部分，你將使用 Gemini API 的嵌入端點為每則文字產生嵌入向量。若想了解更多關於嵌入的資訊，請參閱 [嵌入指南](https://ai.google.dev/docs/embeddings_guide)。

**注意**：嵌入向量的產生是逐筆進行，因此大批量資料會花較多時間！

### 任務類型

`text-embedding-004` 模型支援一個任務類型參數，可以產生適合特定任務的嵌入向量。

任務類型 | 說明
--- | ---
RETRIEVAL_QUERY | 指定輸入文字用於搜尋/檢索查詢。
RETRIEVAL_DOCUMENT | 指定輸入文字為檢索用途的文件。
SEMANTIC_SIMILARITY | 指定輸入文字用於語意文本相似度（STS）。
CLASSIFICATION | 指定嵌入向量將用於分類任務。
CLUSTERING | 指定嵌入向量將用於分群任務。
FACT_VERIFICATION | 指定輸入文字用於事實驗證。

本例中我們將進行分類任務。

In [13]:
from google.api_core import retry
import tqdm
from tqdm.rich import tqdm as tqdmr
import warnings

# 為 Pandas 加入 tqdm 支援
tqdmr.pandas()
# 關閉 tqdm 的實驗性警告
warnings.filterwarnings("ignore", category=tqdm.TqdmExperimentalWarning)

# 定義一個輔助函式，在每分鐘額度不足時進行重試
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

@retry.Retry(predicate=is_retriable, timeout=300.0)
def embed_fn(text: str) -> list[float]:
    # 因為進行分類，所以設定 task_type 為 "classification"
    response = client.models.embed_content(
        model="models/text-embedding-004",
        contents=text,
        config=types.EmbedContentConfig(task_type="classification"),
    )
    return response.embeddings[0].values

def create_embeddings(df):
    df["Embeddings"] = df["Text"].progress_apply(embed_fn)
    return df

In [14]:
df_train = create_embeddings(df_train)
df_test = create_embeddings(df_test)

/Users/yilintsai/anaconda3/envs/ai_env/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/yilintsai/anaconda3/envs/ai_env/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

這段程式碼是為了清晰易讀而優化的，並不是特別快速。讀者可以自行練習實作批次處理[batch](https://ai.google.dev/api/embeddings#method:-models.batchembedcontents)或平行／非同步的向量嵌入產生。執行這個步驟會花上一些時間。

In [15]:
df_train.head()

,Text,Label,Class Name,Encoded Label,Embeddings
1100,Re: Secret algorithm [Re: Clipper Chip and cry...,11,sci.crypt,0,"[-0.0018313533, 0.020108812, -0.03654417, 0.06..."
1101,"Re: Once tapped, your code is no good any more...",11,sci.crypt,0,"[-0.0012209155, 0.009742465, -0.059188582, 0.0..."
1102,"Re: Once tapped, your code is no good any more...",11,sci.crypt,0,"[0.0045050536, 0.020362409, -0.061563283, 0.00..."
1103,"Screw the people, crypto is for hard-core hack...",11,sci.crypt,0,"[0.00011560278, 0.0058189007, -0.047523465, 0...."
1104,Re: Fifth Amendment and Passwords\n\n>>I am po...,11,sci.crypt,0,"[-0.0034216328, 0.028405854, -0.050832406, 0.0..."


## 建立分類模型

接下來，我們將定義一個簡單的模型，此模型接受原始嵌入向量作為輸入，經過一個隱藏層後，輸出各分類的機率。預測結果將代表該篇文章屬於某個新聞分類的機率。

Keras 會自動處理資料的洗牌、計算指標及其他機器學習的基本流程。


In [16]:
import keras
from keras import layers

def build_classification_model(input_size: int, num_classes: int) -> keras.Model:
    return keras.Sequential([
        layers.Input([input_size], name="embedding_inputs"),
        layers.Dense(input_size, activation="relu", name="hidden"),
        layers.Dense(num_classes, activation="softmax", name="output_probs"),
    ])


In [17]:
# 從資料中觀察嵌入向量的大小，也可透過 embed_content 的 output_dimensionality 參數指定
embedding_size = len(df_train["Embeddings"].iloc[0])

classifier = build_classification_model(embedding_size, len(df_train["Class Name"].unique()))
classifier.summary()

classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_probs (Dense)            │ (None, 4)              │         3,076 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 593,668 (2.26 MB)

 Trainable params: 593,668 (2.26 MB)

 Non-trainable params: 0 (0.00 B)

## 訓練模型

最後，我們可以訓練模型。此段程式碼使用早停機制，當準確度穩定時便提前結束訓練，所以實際訓練的 epoch 數可能會少於指定值。


In [18]:
import numpy as np

NUM_EPOCHS = 20
BATCH_SIZE = 32

# 分離訓練與驗證資料的 x 與 y
y_train = df_train["Encoded Label"]
x_train = np.stack(df_train["Embeddings"])
y_val = df_test["Encoded Label"]
x_val = np.stack(df_test["Embeddings"])

# 當準確度穩定時提前停止訓練
early_stop = keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)

# 訓練模型
history = classifier.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_val, y_val),
    callbacks=[early_stop],
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3152 - loss: 1.3622 - val_accuracy: 0.5800 - val_loss: 1.2570
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8085 - loss: 1.1932 - val_accuracy: 0.8400 - val_loss: 1.0904
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8549 - loss: 1.0105 - val_accuracy: 0.7900 - val_loss: 0.9345
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8832 - loss: 0.7984 - val_accuracy: 0.8500 - val_loss: 0.7686
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9271 - loss: 0.6371 - val_accuracy: 0.9200 - val_loss: 0.6144
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9607 - loss: 0.4784 - val_accuracy: 0.8900 - val_loss: 0.5133
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9556 - loss: 0.3636 - val_accuracy: 0.8300 - val_loss: 0.4842
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9623 - loss: 0.3047 - val_accuracy: 0.9000 - v

## 評估模型效能

使用 Keras 的 <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate"><code>Model.evaluate</code></a> 方法，計算測試資料集上的損失值與準確率。


In [19]:
classifier.evaluate(x=x_val, y=y_val, return_dict=True)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9644 - loss: 0.1590


{'accuracy': 0.949999988079071, 'loss': 0.1935824155807495}

更多關於如何使用 Keras 訓練模型及視覺化訓練指標的資訊，請參閱 [使用內建方法進行訓練與評估](https://www.tensorflow.org/guide/keras/training_with_built_in_methods)。

## 嘗試自訂預測

在模型訓練完成且取得不錯的評估結果後，你可以嘗試對新輸入的文字進行預測。使用提供的範例，或自行輸入文字，來檢視模型的表現。


In [20]:
def make_prediction(text: str) -> list[float]:
    """根據提供的文字進行分類預測。"""
    # 由於模型接受嵌入向量作為輸入，先對文字進行嵌入計算。
    embedded = embed_fn(text)
    # 輸入必須為批次形式，這裡用列表包起來表示一筆資料。
    inp = np.array([embedded])
    # 預測結果為批次形式，取出第一筆
    [result] = classifier.predict(inp)
    return result


In [24]:
# 範例文字：此例避免使用太多領域專用的術語，以檢測模型是否避免針對特定術語產生偏誤。
new_text = """
First-timer looking to get out of here.

Hi, I'm writing about my interest in travelling to the outer limits!

What kind of craft can I buy? What is easiest to access from this 3rd rock?

Let me know how to do that please.
"""

result = make_prediction(new_text)

for idx, category in enumerate(df_test["Class Name"].cat.categories):
    print(f"{category}: {result[idx] * 100:0.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
sci.crypt: 0.03%
sci.electronics: 0.10%
sci.med: 0.02%
sci.space: 99.86%


## 進一步閱讀

若想深入了解如何使用 Keras 訓練自訂模型，請參閱 [Keras 指南](https://keras.io/guides/).

*- [Mark McD](https://linktr.ee/markmcd)*

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.